# Índice

- [Machine Learning con Spark](#ml)

    - [Spark.ml](#spark.ml)
        - [Tipos de datos](#data)
        - [Extractores, transformadores y selectores](#data2)
        
    - [Aprendizaje supervisado](#supervisado)
        - [Regresión](#regresion)
            - [Linear Regression](#lr)
            - [Survival Regression](#sr)
            - [Decision Trees](#dt)
            - [Random Forest](#rf)
            - [Gradient Boosted Trees](#gbt)
            - [Isotonic Regression](#riso)
        - [Clasificación](#clasificacion)
            - [Linear Regression](#lrb)
            - [Decision Trees](#dtc)
            - [Gradient Boosted Trees](#gbtc)
            - [Random Forest](#rfc)
            - [Naive Bayes](#nb)
            - [Perceptron](#perceptron)
            
        - [Cross Validation](#cv)
        - [Spark Pipeline](#pipeline)
        
    - [Spark.mllib](#mllib)
    
<div id='xx' />

<div id='ml' />

# Machine Learning con Spark

MLlib es la librería de Spark de aprendizaje automático. Su objetivo es hacer machine learning escalable y sencillo.

Se compone de algortimos comunes de aprendizaje y utilidades, incluyendo clasificación, regresión, clustering, filtros colaborativos, reducción de dimensionalidad asó como de primtivas de optimización de bajo nivel y APIs de flujo de alto nivel.

Está formado por dos paquetes:

- ***Spark.mllib***, que contiene el API original sobre RDDs.

- ***Spark.ml***, proporciona APIs de más alto nivel contruidas sobre dataframes.

<div id='spark.ml' />

## Spark.ml

Los conceptos claves introducidos por el API de [Spark.ml](https://spark.apache.org/docs/latest/ml-guide.html) son los siguientes:

- **Dataframe**: Spark ML usa los dataframes de Spark SQL como los dataset principales para tareas de ML.

- **Transformer**: se trata de un algoritmo que puede transformar un dataframe en otro dataframe, por ejemplo, un modelo de ML es un *Transformer* que transforma un dataframe con variables predictoras en otro con predicciones.

- **Estimator**: es un algoritmo que se puede aplicar sobre un dataframe para producir un *Transformer*. Por ejemplo, un algoritmo de aprendizaje es un *Estimator* que se entrena con un dataframe y produce un modelo.

- **Pipeline**: encadena múltiples *Transformes* y *Estimators* juntos especificando un flujo de trabajo ML.

- **Parameter**: todos los *Transformer* y *Estimators* comparten ahora un API para especificar los parámetros.

Como salida a la aplicación de cada uno de los algoritmos se obtiene un objeto tipo *model* de la clase correspondiente. Por ejemplo, si aplicamos LinearRegresion obtenemos LineaRegresionModel.

<div id='data' />

### Tipos de datos especiales

Las librerías ml y mllib soportan los siguientes tipos de datos especiales:

- **Vectores locales** que pueden ser:

    - **Vectores densos (dense)** en cuyo caso se especifican todos los valores del vector y se pueden crear a partir de arrays numpy y listas python.
    
    - **Vectores dispersos (sparse)** aquellos que contienen una gran cantidad de ceros, por lo que se definen especificando únicamente los valores no nulos junto con sus posiciones. Se pueden crear a partir de la clase *SparseVector* de mllib y *csc_matrix* de scipy. 

In [1]:
from pyspark.mllib.linalg import Vectors

# Distintas formas de crear un mismo vector.

vdense = Vectors.dense([0, 1.0, 0, 5.5])
print(vdense)

# Vecotors.sparse se indica la longitud y posteriormente las posiciones con sus respectivos valores.

vsparse1 = Vectors.sparse(4, {1: 1.0, 3: 5.5})
print(vsparse1)

vsparse2 = Vectors.sparse(4, [(1, 1.0), (3, 5.5)])
print(vsparse2)

vsparse3 = Vectors.sparse(4, [1, 3], [1.0, 5.5])
print(vsparse3)

[0.0,1.0,0.0,5.5]
(4,[1,3],[1.0,5.5])
(4,[1,3],[1.0,5.5])
(4,[1,3],[1.0,5.5])


Para la manipulación de vectores se recomienda utilizar los comandos implementados en la clase *Vectors*:

- **Labeled Points**: vectores locales (densos o dispersos) que tienen asociados una respuesta o etiqueta numérica que a menudo suelen representar una predicción. Pueden ser de los siguientes tipos:
    
    - **Double** en casos de regresión.
    
    - **0/1** en casos de clasificación binaria.
      
    - **Índices numéricos** en caso de clasificación multiclase.
    
Los vectores se utilizan normalmente con la librería spark.ml mientras que los puntos etiquetados suelen ser más utilizados con spark.mllib

In [2]:
#from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

# Creación a partir de un dense vector.

ld = LabeledPoint(1.0, [1.0, 0.0, 3.0])
print(ld)

# Creación a partir de un sparse vector.

ls = LabeledPoint(1.0, Vectors.sparse(3, [0, 1.0], [2, 3.0]))
print(ls)

(1.0,[1.0,0.0,3.0])
(1.0,(3,[0,1],[2.0,3.0]))


Además de estos dos tipos principales existen las **Matrices Locales** densas y/o dispersas y las **Matrices Distribuidas** que a su vez se diferencian en:

    - Row Matrix
    - IndexedRowMatrix
    - CoordinateMatrix
    - Block Matrix

<div id='data2' />

#### Preparación de datos: extractores, transformadores y selectores

Existen distintos componentes que permiten realizar múltiples tareas sobre los dataframe que ayudan a preparar los datos. Hay tres tipos: extractores, transformadores, selectores.

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *

sc = SparkContext()
spark = SparkSession(sc)

# Tipado de datos: DoubleType

fields = [StructField("ID", IntegerType(), True), StructField("sepal_length", DoubleType(), True), StructField("sepal_width", DoubleType(), True), StructField("petal_length", DoubleType(), True), StructField("petal_width", DoubleType(), True), StructField("specie", StringType(), True)]
schema = StructType(fields)

iris_csvDF = spark.read.format("csv").options(header = True).load("../data/iris.csv", schema = schema)
iris_csvDF.show(2)

+---+------------+-----------+------------+-----------+-----------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|
+---+------------+-----------+------------+-----------+-----------+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
+---+------------+-----------+------------+-----------+-----------+
only showing top 2 rows



In [4]:
from pyspark.ml.feature import VectorAssembler as VA

# Construimos un assmebler para unir todas las columnas predictoras del dataset en un vector de features.

v = VA(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
d = v.transform(iris_csvDF)
d.show(2)

+---+------------+-----------+------------+-----------+-----------+-----------------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|         features|
+---+------------+-----------+------------+-----------+-----------+-----------------+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
+---+------------+-----------+------------+-----------+-----------+-----------------+
only showing top 2 rows



<div id='supervisado' />

## Aprendizaje supervisado

<div id='regresion' />

### Regresión

Spark ML implementa varios algoritmos para realizar regresiones:

<div id='lr' />

#### Regresión lineal (Linear Regression)

`LinearRegression?` soporta múltiples tipos de regularización dependiendo del valor del parámetro `elasticNetParam` y `regParam`:

- **Ordinary Least Squares (OLS)** (ninguna regularización): `regParam = 0`
    
- **L2 (ridge regression)**: `elasticNetParam = 0, regParam > 0`

- **L1 (lasso)**: `elasticNetParam = 1, regParam > 0`

- **L2+L1 (elastic net)**: `0 < elasticNetParam < 1, regParam > 0`

<div id='sr' />

#### Regresión de supervivencia (Survival Regression)

La clase spark que utilizamos para este tipo de modelos es `AFTSurvivalRegression?`. Esta clase implementa el modelo [*AFT (Accelerate Failure Time)*](https://en.wikipedia.org/wiki/Accelerated_failure_time_model) basado en la distribución de Weibull de tiempo de supervivencia.

<div id='dt' />

#### Regresión con árboles de desición (Decision Trees)

Este modelo viene con la clase `DecisionTreeRegressor?`. Soporta variables predictoras de tipo *continuo* o *categóricas*.

<div id='rf' />

#### Regresión con Random Forest

`RandomForestRegressor?` es la clase de spark para este tipo de algoritmos. Al igual que Dessicion Tree también soporta variables predictoras de tipo *continuo* o *categórica*.

<div id='gbt' />

#### Regresión GBT (Gradient Boosted Trees)

Este modelo se implementa en la clase `GBTRegressor?`. Soporta variables predictoras de tipo *continuo* o *categórica*.

<div id='riso' />

#### Regresión isotónica

La clase spark correspondiente es `IsotonicRegression?`. [*Isotonic Regression*](https://en.wikipedia.org/wiki/Isotonic_regression) se encuentra en fase experimental en spark.ml y sólo admite una única variable predictora.

In [5]:
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler as VA
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import IsotonicRegression

from pyspark.ml.evaluation import RegressionEvaluator

# Carga de datos.

df = spark.read.format("csv").options(header=True, inferSchema=True).load("../data/faithful.csv")

# Creamos un VectorAssembler (vector de características) para tener todas las variables predictoras en un vector features.

vAssem = VA(inputCols=["waiting"], outputCol="features")
vt = vAssem.transform(df)
vt.show(3)

# Creamos conjunto entrenamiento y test.

splits = vt.randomSplit([0.7, 0.3], 1234)
train = splits[0]
test = splits[1]

# Definimos los algoritmos de regresión indicando la columna target=eruptions.
# Aplicamos el modelo sobre el conjunto de entrenamiento.

lr = LinearRegression(maxIter=5, regParam=0.0, solver="normal", labelCol="eruptions", featuresCol="features")
lr_model = lr.fit(train)

#aft = AFTSurvivalRegression(maxIter=5, labelCol="eruptions", featuresCol="features")
#aft_model = aft.fit(train)

dt = DecisionTreeRegressor(maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0, labelCol="eruptions", featuresCol="features", checkpointInterval=10)
dt_model = dt.fit(train)

rf = RandomForestRegressor(maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0, labelCol="eruptions", featuresCol="features")
rf_model = rf.fit(train)

gbt = GBTRegressor(maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0, labelCol="eruptions", featuresCol="features", checkpointInterval=10, lossType="squared")
gbt_model = gbt.fit(train)

ir = IsotonicRegression(labelCol="eruptions", featuresCol="features", isotonic=True)
ir_model = ir.fit(train)

# Obtenemos algunos parámetros del modelo.

print("Intercept: {0}".format(lr_model.intercept))
print("Coefficients: {0}".format(lr_model.coefficients))
print(" ")

# Hacemos predicciones sobre el conjunto de test.

predict_train = lr_model.transform(train)
predict_test = lr_model.transform(test)
predict_test.show(5)

# Usamos un evaluador para conocer el error de entrenamiento y test.

ev = RegressionEvaluator(labelCol="eruptions")
print("RMSE test: {0}".format(ev.evaluate(predict_test, {ev.metricName: "rmse"})))
print("RMSE training: {0}".format(ev.evaluate(predict_train, {ev.metricName: "rmse"})))

trainingSummary = lr_model.summary
print("RMSE training: %f" % trainingSummary.rootMeanSquaredError)
print("R2 training: %f" % trainingSummary.r2)

# Lista de parámetros

#lr_model.params

+---+---------+-------+--------+
|_c0|eruptions|waiting|features|
+---+---------+-------+--------+
|  1|      3.6|     79|  [79.0]|
|  2|      1.8|     54|  [54.0]|
|  3|    3.333|     74|  [74.0]|
+---+---------+-------+--------+
only showing top 3 rows

Intercept: -1.9156822121843768
Coefficients: [0.07661202567343066]
 
+---+---------+-------+--------+------------------+
|_c0|eruptions|waiting|features|        prediction|
+---+---------+-------+--------+------------------+
|  1|      3.6|     79|  [79.0]| 4.136667816016645|
|  2|      1.8|     54|  [54.0]|2.2213671741808785|
|  3|    3.333|     74|  [74.0]|3.7536076876494917|
| 14|     1.75|     47|  [47.0]|1.6850829944668642|
| 27|    1.967|     55|  [55.0]| 2.297979199854309|
+---+---------+-------+--------+------------------+
only showing top 5 rows

RMSE test: 0.45981889015866745
RMSE training: 0.5079462531939202
RMSE training: 0.507946
R2 training: 0.796555


<div id='clasificacion' />

### Clasificación

<div id='lrb' />

#### Clasificación mediante regresión logística (binaria)

`LogisticRegression?` sólo permite clasificaciones binarias.

In [6]:
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler as VA
#from pyspark.sql.typer import *

from pyspark.ml.classification import LogisticRegression

from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Nos interesa predecir la variable admit, la cual nos dice si un alumno
# es admitido en una escuela superior. Las otras variables son:
# gre (Graduate Record Exam scores)
# gpa (Grade Point Average)
# rank: prestigio de la escuela de donde viene el alumno.

# Nos aseguramos de que el target sea DoubleType aunque el modelo también funciona si definimos este campo de tipo Binary o Boolean

ad_schema = StructType([StructField("admit", DoubleType(), True),
                        StructField("gre", DoubleType(), True), 
                        StructField("gpa", DoubleType(), True),
                        StructField("rank", StringType(), True)])

data = spark.read.format("csv").options(header=True, inferSchema=True).load("../data/binary.csv", schema=ad_schema)

# Transformamos la variable rank en una lista de valores numéricos.

indexer = StringIndexer(inputCol="rank", outputCol="ranknum")
i_model = indexer.fit(data)
i_data = i_model.transform(data)

# Transformamos la columna con la variable categórica ranknum en variables dummies.

encoder = OneHotEncoder(inputCol="ranknum", outputCol="dummy_rank", dropLast=False)
dum_data = encoder.transform(i_data)

# Construimos el assembler para unir todas las columnas predictoras en un vector features.

vAssem2 = VA(inputCols=["gre", "gpa", "dummy_rank"], outputCol="features")
vt2 = vAssem2.transform(dum_data)
vt2.show(3)

# Creamos conjunto entrenamiento y test.

splits2 = vt2.randomSplit([0.7, 0.3], 1234)
train2 = splits2[0]
test2 = splits2[1]

# Definimos los algoritmos de clasificación.
# Aplicamos el modelo sobre el conjunto de entrenamiento.

log_rg = LogisticRegression(maxIter=5, regParam=0.0, labelCol="admit")
log_rg_model = log_rg.fit(train2)

# Obtenemos algunos parámetros del modelo.

print("Intercept: {0}".format(log_rg_model.intercept))
print("Coefficients: {0}".format(log_rg_model.coefficients))
print(" ")

# Hacemos predicciones sobre el conjunto de test.

predict_train2 = log_rg_model.transform(train2)
predict_test2 = log_rg_model.transform(test2)
predict_test2.show(5)

# Evaluamos el resultado mostrando el área bajo la curva ROC

ev2 = BinaryClassificationEvaluator(labelCol="admit")
print("Área bajo curva ROC: {0}".format(ev2.evaluate(predict_test2, {ev2.metricName: "areaUnderROC"})))

+-----+-----+----+----+-------+-------------+--------------------+
|admit|  gre| gpa|rank|ranknum|   dummy_rank|            features|
+-----+-----+----+----+-------+-------------+--------------------+
|  0.0|380.0|3.61|   3|    1.0|(4,[1],[1.0])|[380.0,3.61,0.0,1...|
|  1.0|660.0|3.67|   3|    1.0|(4,[1],[1.0])|[660.0,3.67,0.0,1...|
|  1.0|800.0| 4.0|   1|    3.0|(4,[3],[1.0])|[800.0,4.0,0.0,0....|
+-----+-----+----+----+-------+-------------+--------------------+
only showing top 3 rows

Intercept: -0.8943498051582796
Coefficients: [0.0017865683210316042,-0.24545377013623773,0.0440126119321804,-0.5289020158341485,-0.7855737230162501,0.8250528991613559]
 
+-----+-----+----+----+-------+-------------+--------------------+--------------------+--------------------+----------+
|admit|  gre| gpa|rank|ranknum|   dummy_rank|            features|       rawPrediction|         probability|prediction|
+-----+-----+----+----+-------+-------------+--------------------+--------------------+---------

<div id='dtc' />

#### Clasificación con árboles de desición (Decision Trees)

También se puede utilizar este algoritmo para problemas de clasificación. La clase correspondiente es `DecisionTreeClassifier?`. Soporta variables predictoras de tipo *continuo* o *categóricas* y predicciones *multiclase*.

<div id='gbtc' />

#### Clasificador GBT

La clase python que utilizamos para la implementación de modelos de clasificación con *árboles GBT* es `GBTClassifier?`. Soporta variables predictoras de tipo *continuo* o *categóricas* y realiza predicciones sobre variables binarias (no soporta clasificación *multiclase*).

<div id='rfc' />

#### Clasificador Random Forest

`RandomForestClassifier?` es la clase que aplica este modelo. Soporta variables predictoras de tipo *continuo* o *categóricas* y predicciones *multiclase*.

In [7]:
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler as VA
#from pyspark.sql.typer import *

from pyspark.ml.classification import RandomForestClassifier

#from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Clasificación del dataset iris.csv con RandomForest
# Usaremos el dataset iris_csvDF cargado anteriormente.

# Primero transformamos la etiqueta specie de string a double en varios niveles.

indexer2 = StringIndexer(inputCol="specie", outputCol="specienum")
i2_model = indexer2.fit(iris_csvDF)
i2_data = i2_model.transform(iris_csvDF)

# Construimos un assembler para unir todas las columnas predictoras del dataset en un vector de features.

vAssem3 = VA(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
vt3 = vAssem3.transform(i2_data)
vt3.show(3)

# Creamos conjunto entrenamiento y test.

splits3 = vt3.randomSplit([0.7, 0.3], 1234)
train3 = splits3[0]
test3 = splits3[1]

# Definimos el algoritmo de RandomForest.
# Aplicamos el modelo sobre el conjunto de entrenamiento.

rfc = RandomForestClassifier(numTrees=20, maxDepth=2, labelCol="specienum", seed=42)
rfc_model = rfc.fit(train3)

# Hacemos predicciones sobre el conjunto de test.

predict_train3 = rfc_model.transform(train3)
predict_test3 = rfc_model.transform(test3)
predict_test3.show(5)

# Aplicamos el evaluador multiclase para ver la eficiencia del modelo.

ev3 = MulticlassClassificationEvaluator(labelCol="specienum")
#print("Precision: {0}, Recall: {1}, F1: {2}".format(ev3.evaluate(predict_test3, {ev3.metricName: "precision"}), 
#                                                    ev3.evaluate(predict_test3, {ev3.metricName: "recall"}), 
#                                                    ev3.evaluate(predict_test3, {ev3.metricName: "f1"})))

print("Accuracy: %g" % (ev3.evaluate(predict_test3, {ev3.metricName: "accuracy"})))
print("Precision: %g" % (ev3.evaluate(predict_test3, {ev3.metricName: "weightedPrecision"})))
print("Recall: %g" % (ev3.evaluate(predict_test3, {ev3.metricName: "weightedRecall"})))
print("F1: %g" % (ev3.evaluate(predict_test3, {ev3.metricName: "f1"})))

+---+------------+-----------+------------+-----------+-----------+---------+-----------------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|specienum|         features|
+---+------------+-----------+------------+-----------+-----------+---------+-----------------+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|      0.0|[5.1,3.5,1.4,0.2]|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|      0.0|[4.9,3.0,1.4,0.2]|
|  3|         4.7|        3.2|         1.3|        0.2|Iris-setosa|      0.0|[4.7,3.2,1.3,0.2]|
+---+------------+-----------+------------+-----------+-----------+---------+-----------------+
only showing top 3 rows

+---+------------+-----------+------------+-----------+-----------+---------+-----------------+--------------------+--------------------+----------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|specienum|         features|       rawPrediction|         probability|prediction|
+---+

<div id='nb' />

#### Clasificador NaiveBayes

Este modelo se implementa en python con la clase `NaiveBayes?` y soporta los tipos *Multinomial NB* y *Bernoulli NB*.

El multinomial puede manejar de forma eficiente variables discretas tipo vectores TF-IDF de palabras de documentos para clasificación de textos.

El de bernoulli funciona de forma similar al multinomial pero transformando el valor TF-IDF en ceros y unos (0/1) indicando la presencia o no de la palabra correspondiente. Una condición más para este modelo es que los valores de los predictores no pueden ser negativos.

<div id='perceptron' />

#### Clasificador Perceptrón Multicapa

Este modelo viene con la clase `MultilayerPerceptronClassifier?` y presenta las siguientes características:

- Cada capa cuenta con una función de activación sigmoide.
- La capa de salida tiene **softmax**.
- El número de entradas tiene que ser igual al tamaño del vector de predictores *features*.
- El número de salidas tiene que ser igual al número total de variables target *labels*.

<div id='cv' />

### Validación cruzada (Cross Validation)

Usaremos el modelo de regresión lineal sobre el dataset *faithful.csv* como ejemplo de validación cruzada. Las diferencias surgen cuando llegamos a la parte de creación del modelo:

1) Carga de datos, creación del vector de características (VectorAssembler) y creación de conjunto entrenamiento y test.

2) Creación del modelo LinearRegression donde solo pasamos como parámetro la variable target.

3) Ajuste con k-fold y grid de parámetros: `regParam` y `elasticNetParam` se pasan al modelo a un objeto de tipo grid en el que se indican los conjuntos de valores que tomarán dichos parámetros iterando en diferentes ejecuciones hasta que se encuentre el valor más óptimo. Es decir, si vemos que el parámetro `regParam` viene acompañado por `[0.0, 0.01, 0.05, 0.5]` significa que habrá una ejecución por cada uno de los valores de la lista. Lo mismo para `elasticNetParam`. Por tanto se realizarán las ejecuciones necesarias para cubrir las distintas combinaciones de ambos parámetros.

4) Se crea un objeto tipo `CrossValidator` al que se le pasa como parámetro el modelo, el grid de parámetros, el evaluador (que servirá para ir comparando los modelos y quedarse con el mejor) y el número de iteraciones en el que se irán cambiando los conjuntos de entrenamiento y test mediante cross validation (kfold) que en este caso serán 5 `numFolds = 5`. Así pues, habrá 5 ejecuciones por cada par de valores de `regParam` y `elasticNetParam`, lo que hará subir el número de trabajos ejecutados por Spark.
    
5) Se aplica la función `fit()` generándose un modelo tipo `CrossValidationModel`. De este tipo de objeto no se pueden obtener el *intercept* y los *coefficients* sino que hay que acceder a su objeto hijo `bestModel` que es de tipo `LinearRegressionModel` y que es el modelo lineal optimizado resultado de todo el proceso iterativo.

6) Para realizar predicciones se pueden utilizar el modelo de tipo `CrossValidationModel` o el que corresponde a `bestModel`

7) Finalmente hacemos una evaluación del modelo

In [8]:
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.feature import VectorAssembler as VA
#from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
#from pyspark.ml.evaluation import RegressionEvaluator

# Carga de datos.
#df = spark.read.format("csv").options(header=True, inferSchema=True).load("data/faithful.csv")

# Creamos un VectorAssembler (vector de características) para tener todas las variables predictoras en un vector features.
#vAssem = VA(inputCols=["waiting"], outputCol="features")
#vt = vAssem.transform(df)
#vt.show(3)

# Creamos conjunto entrenamiento y test.
#splits = vt.randomSplit([0.7, 0.3], 1234)
#train = splits[0]
#test = splits[1]

# Creación del modelo LinearRegression. Solo pasamos como parámetro la variable target.
# Ajuste con k-fold y grid de parámetros.

lrcv = LinearRegression(labelCol="eruptions")
lr_grid = ParamGridBuilder().addGrid(lrcv.regParam, [0.0, 0.01, 0.05, 0.5]).addGrid(lrcv.elasticNetParam, [0.0, 0.5, 1.0]).build()
lr_ev = RegressionEvaluator(labelCol="eruptions")
cv = CrossValidator(estimator=lrcv, estimatorParamMaps=lr_grid, evaluator=lr_ev, numFolds=5)
cv_model = cv.fit(train)

print("Modelo cv_model tipo: ", type(cv_model.bestModel))
print("Modelo cv_model.bestModel tipo: ", type(cv_model.bestModel))

print(" ")

# Obtenemos algunos parámetros del modelo.

print("Intercept: {0}".format(cv_model.bestModel.intercept))
print("Coefficients: {0}".format(cv_model.bestModel.coefficients))
print(" ")

# Hacemos predicciones sobre el conjunto de test.

predict_cv_train = cv_model.transform(train)
predict_cv_test = cv_model.transform(test)
predict_cv_test.show(3)

print("RMSE test: %g" % (lr_ev.evaluate(predict_cv_test, {lr_ev.metricName: "rmse"})))
print("RMSE training: %g" % (lr_ev.evaluate(predict_cv_train, {lr_ev.metricName: "rmse"})))


Modelo cv_model tipo:  <class 'pyspark.ml.regression.LinearRegressionModel'>
Modelo cv_model.bestModel tipo:  <class 'pyspark.ml.regression.LinearRegressionModel'>
 
Intercept: -1.9156822121843768
Coefficients: [0.07661202567343066]
 
+---+---------+-------+--------+------------------+
|_c0|eruptions|waiting|features|        prediction|
+---+---------+-------+--------+------------------+
|  1|      3.6|     79|  [79.0]| 4.136667816016645|
|  2|      1.8|     54|  [54.0]|2.2213671741808785|
|  3|    3.333|     74|  [74.0]|3.7536076876494917|
+---+---------+-------+--------+------------------+
only showing top 3 rows

RMSE test: 0.459819
RMSE training: 0.507946


<div id='pipeline' />

### Flujos ejecución Spark Pipeline

Con Spark ML existe el concepto de *Pipeline* que representa el flujo de trabajo que encadena varios *Transformers* y *Estimators*.

A continuación podemos ver como en un mismo **pipeline** se indican como pasos (stages) los transformers y estimators (indexer, vectorAssembler y rf) a ejecutar y el orden en que queremos que se haga. Por último se crea el modelo `pipeModel` aplicando mediante la función `fit()` el pipeline al conjunto de entrenamiento.

In [9]:
#from pyspark.ml.feature import VectorAssembler as VA
#from pyspark.ml.classification import RandomForestClassifier
#from pyspark.ml.feature import StringIndexer
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline


# Clasificación del dataset iris.csv con RandomForest mediante Pipeline.
# Usaremos el dataset iris_csvDF cargado anteriormente.

# Primero transformamos la etiqueta specie de string a double en varios niveles.
indexer_pip = StringIndexer(inputCol="specie", outputCol="specienum")

# Construimos un assembler para unir todas las columnas predictoras del dataset en un vector de features.
vAssem_pip = VA(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")

# Creamos conjunto entrenamiento y test.

splits_pip = iris_csvDF.randomSplit([0.7, 0.3], 1234)
train_pip = splits_pip[0]
test_pip = splits_pip[1]


# Definimos el algoritmo de RandomForest.
rfc_pip = RandomForestClassifier(numTrees=20, maxDepth=2, labelCol="specienum", seed=42)

# Definimos el Pipeline con las operaciones en orden.
pipeline = Pipeline(stages= [indexer_pip, vAssem_pip, rfc_pip])

# Creamos el modelo aplicandoel pipeline al conjunto de entrenamiento.
pipeline_model = pipeline.fit(train_pip)

# Hacemos predicciones sobre el conjunto de test.

predict_train_pip = pipeline_model.transform(train_pip)
predict_test_pip = pipeline_model.transform(test_pip)
predict_test3.show(5)

#print("Accuracy: %g" % (ev3.evaluate(predict_test3, {ev3.metricName: "accuracy"})))


+---+------------+-----------+------------+-----------+-----------+---------+-----------------+--------------------+--------------------+----------+
| ID|sepal_length|sepal_width|petal_length|petal_width|     specie|specienum|         features|       rawPrediction|         probability|prediction|
+---+------------+-----------+------------+-----------+-----------+---------+-----------------+--------------------+--------------------+----------+
|  1|         5.1|        3.5|         1.4|        0.2|Iris-setosa|      0.0|[5.1,3.5,1.4,0.2]|[19.9555555555555...|[0.99777777777777...|       0.0|
|  2|         4.9|        3.0|         1.4|        0.2|Iris-setosa|      0.0|[4.9,3.0,1.4,0.2]|[19.9555555555555...|[0.99777777777777...|       0.0|
|  3|         4.7|        3.2|         1.3|        0.2|Iris-setosa|      0.0|[4.7,3.2,1.3,0.2]|[19.9555555555555...|[0.99777777777777...|       0.0|
| 14|         4.3|        3.0|         1.1|        0.1|Iris-setosa|      0.0|[4.3,3.0,1.1,0.1]|[19.9555555

<div id='mllib' />

## Spark.mllib

[Spark.mllib](https://spark.apache.org/docs/latest/mllib-guide.html) es una librería alternativa para hacer machine learning sobre RDDs. Vamos a ver un ejemplo con un modelo RandomForest para comparar con los resultados anteriores. En este [enlace](https://spark.apache.org/docs/latest/mllib-ensembles.html#random-forests) tpodemos ver la guía del modelo. Los puntos distintos más relevantes de mllib con respecto a ml son los siguientes:

- Se utilizan RDDs en vez de dataframes y *Labeled Points* en vez de *Vectors*.

- El modelo se crea de una pasada (no es necesario aplicar el estimador con la función `fit()`.

- Para predecir se utiliza la función `predict()` en vez de `transform()`.

In [10]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest

# Creamos una función para pasar las etiquetas a numérico.

def getLabelNum(label):
    if label == "Iris-virginica":
        return 0.0
    elif label == "Iris-setosa":
        return 1.0
    else:
        return 2.0

# Cargamos datos y eliminamos cabeceras

rdd = sc.textFile("../data/iris.csv")

headers = rdd.first()
rdd = rdd.filter(lambda line: line != headers)

rdd = rdd.map(lambda x: x.split(","))

# Metemos cada uno de los campos separados por coma en un punto etiquetado.

rdd = rdd.map(lambda x: LabeledPoint(getLabelNum(x[5]), [x[0], x[1], x[2], x[3], x[4]]))
print(rdd.take(3))

# Dividimos conjunto de datos en entrenamiento y test

(trainingData, testData) = rdd.randomSplit([0.7, 0.3])

# Creamos modelo RandomForest.

rdd_model = RandomForest.trainClassifier(trainingData, numClasses=3, categoricalFeaturesInfo={}, numTrees=1, maxDepth=2, seed=42)

print(" ")
print("Número de nodos de los árboles: {0}".format(rdd_model.totalNumNodes()))
print(" Modelo RandomForest de entrenamiento: ")
print(rdd_model.toDebugString())

# Creamos un RDD sobre el que hacer predicciones. Hacemos predicciones sobre conjunto test.

predictions = rdd_model.predict(testData.map(lambda x: x.features))
print("Predicción sobre test: {0}".format(predictions.collect()))

labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())

print(" ")
print("Test Error = " + str(testErr))

[LabeledPoint(1.0, [1.0,5.1,3.5,1.4,0.2]), LabeledPoint(1.0, [2.0,4.9,3.0,1.4,0.2]), LabeledPoint(1.0, [3.0,4.7,3.2,1.3,0.2])]
 
Número de nodos de los árboles: 5
 Modelo RandomForest de entrenamiento: 
TreeEnsembleModel classifier with 1 trees

  Tree 0:
    If (feature 0 <= 102.5)
     If (feature 3 <= 2.45)
      Predict: 1.0
     Else (feature 3 > 2.45)
      Predict: 2.0
    Else (feature 0 > 102.5)
     Predict: 0.0

Predicción sobre test: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 
Test Error = 0.02702702702702703
